In [1]:
import argparse
import logging
import numpy as np
import os
import re
import setproctitle
import skimage
import skimage.io
import skimage.transform
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import sys
sys.path.append('/home/lupin/PhotoEnhance/hdrnet_lupin/hdrnet')
import models as models
import data_pipeline_lupin as dp
import metrics as metrics
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

In [2]:
logging.basicConfig(format="[%(process)d] %(levelname)s %(filename)s:%(lineno)s | %(message)s")
log = logging.getLogger("train")
log.setLevel(logging.INFO)

In [3]:
def main(args,model_params):
    setproctitle.setproctitle('hdrnet_run')
    data_pipe = getattr(dp, args.data_pipeline)
    mdl = getattr(models, model_params['model_name'])
    with tf.compat.v1.variable_scope('eval_data'):
        eval_data_pipeline = data_pipe(
          args.input,
          shuffle=False,
          batch_size=1, nthreads=1,
          fliplr=False, flipud=False, rotate=False,
          random_crop=False,
          output_resolution = args.output_resolution)
    eval_samples = eval_data_pipeline.samples
    with tf.compat.v1.variable_scope('inference'):
        eval_prediction = mdl.inference(
            eval_samples['lowres_input'], eval_samples['image_input'],
            model_params, is_training=False)
#     output = tf.cast(255.0*tf.squeeze(tf.clip_by_value(eval_prediction, 0, 1)), tf.uint8)
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True  # Do not canibalize the entire GPU
    sv = tf.compat.v1.train.Supervisor(logdir=args.checkpoint_dir)
    psnr = 0
    ssim = 0
    with sv.managed_session(config=config) as sess:
        while True:
            if sv.should_stop():
                log.info("stopping supervisor")
                break
            try:
                for it in tqdm(range(eval_data_pipeline.nsamples)):
                    prediction, label = sess.run([eval_prediction,eval_samples['image_output']])
                    prediction = np.squeeze(prediction)
                    label = np.squeeze(label)
                    psnr += compare_psnr(prediction,label,data_range=1.0)
                    ssim += compare_ssim(prediction,label,data_range=1.0, multichannel = True)
# #                     fname = os.path.splitext(os.path.basename(input_path))[0]
# #                     fname = input_path.split('/')[-1]
#                     fname = input_path
#                     output_path = os.path.join(args.output, fname)
#                     basedir = os.path.dirname(output_path)
#                     if not os.path.exists(basedir):
#                         os.makedirs(basedir)
#                     skimage.io.imsave(output_path, out_)
                break
            except tf.errors.AbortedError:
                log.error("Aborted")
                break
            except KeyboardInterrupt:
                break
    psnr /= 767
    ssim /= 767
    print(psnr,ssim)

In [4]:
parser = argparse.ArgumentParser()
req_grp = parser.add_argument_group('required')
req_grp.add_argument('--checkpoint_dir', default= '/home/lupin/PhotoEnhance/hdrnet_lupin/hdrnet/checkpoint')
req_grp.add_argument('--input', default='/home/lupin/PhotoEnhance/dataset/SICE/Part2/test.txt')
req_grp.add_argument('--output', default='/home/lupin/PhotoEnhance/hdrnet_lupin/output')
req_grp.add_argument('--lowres_input', default=None, help='path to the lowres, TF inputs')
model_grp = parser.add_argument_group('model_params')
model_grp.add_argument('--model_name', default=models.__all__[0], type=str, help='classname of the model to use.', choices=models.__all__)
model_grp.add_argument('--data_pipeline', default='ImageFilesDataPipeline', help='classname of the data pipeline to use.', choices=dp.__all__)
model_grp.add_argument('--net_input_size', default=256, type=int, help="size of the network's lowres image input.")
model_grp.add_argument('--output_resolution', default=[900, 1200], type=int, nargs=2, help='resolution of the output image.')
model_grp.add_argument('--batch_norm', dest='batch_norm', action='store_true', help='normalize batches. If False, uses the moving averages.')
model_grp.add_argument('--nobatch_norm', dest='batch_norm', action='store_false')
model_grp.add_argument('--channel_multiplier', default=1, type=int,  help='Factor to control net throughput (number of intermediate channels).')
model_grp.add_argument('--guide_complexity', default=16, type=int,  help='Control complexity of the guide network.')
model_grp.add_argument('--luma_bins', default=8, type=int,  help='Number of BGU bins for the luminance.')
model_grp.add_argument('--spatial_bin', default=16, type=int,  help='Size of the spatial BGU bins (pixels).')
parser.set_defaults(batch_norm=True)
args = parser.parse_args(args = [])
model_params = {}
for a in model_grp._group_actions:
    model_params[a.dest] = getattr(args, a.dest, None)
main(args,model_params)
#17.400723839855633 0.5410175975751349

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


[22315] WARNING deprecation.py:323 | From /home/lupin/PhotoEnhance/hdrnet_lupin/hdrnet/data_pipeline_lupin.py:184: slice_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


[22315] WARNING deprecation.py:323 | From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:373: range_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


...

INFO:tensorflow:Recording summary at step None.


[22315] INFO supervisor.py:1050 | Recording summary at step None.
 67%|██████▋   | 515/767 [01:59<00:55,  4.53it/s]

INFO:tensorflow:Recording summary at step None.


[22315] INFO supervisor.py:1050 | Recording summary at step None.
100%|██████████| 767/767 [02:58<00:00,  4.30it/s]

17.400723839855633 0.5410175975751349


In [1]:
import torch
import numpy as np

In [3]:
A = torch.randn(3, 3,15,15)
# torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )
B = torch.max(A,1)

In [10]:
B[0].unsqueeze(1).shape

torch.Size([3, 1, 15, 15])

In [11]:
C = torch.mean(A,1)

In [12]:
C.shape

torch.Size([3, 15, 15])